# 비지도 학습 기반 형태소 분석
* 본 파일은 한국어 임베딩 3.2을 공부하며 정리한 자료임을 밝힙니다.

## Reference
* lovit님의 soynlp tutorial을 따라하며 공부한 자료임을 밝혀둡니다.

## NounExtractor
* 통계를 기반으로 단어의 경계를 학습하는 비지도 학습
* Accessor Variety, Branching Entropy, Cohesion Score의 통계를 이용

In [1]:
from soynlp.utils import DoublespaceLineCorpus
from soynlp.word import WordExtractor

corpus_fname = 'C:/Users/sbh0613/Desktop/NLP/ratsgo/my Preprocessing/2016-10-20.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)

train은 substrings의 빈도수를 카운팅 하는 것이며, extract는 init에 들어가는 값을 기준으로 단어를 선택하여 준다.

In [2]:
%%time

from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.680 Gbse memory 0.756 Gb
all cohesion probabilities was computed. # words = 16942
all branching entropies was computed # words = 355061
all accessor variety was computed # words = 355061
Wall time: 2min 16s


## 이제 위에서 얻은 noun들의 scores로 tokenize를 해보자.

soynlp는 세 가지 토크나이저를 제공한다.
* LTokenizer: 띄어쓰기가 잘 되어 있다면 이거 사용. 한국어 어절의 구조를 "명사+조사"처럼 "L + [R]"로 생각한다.
* MaxScoreTokenizer: 띄어쓰기가 제대로 지켜지지 않은 데이터에 사용.
* RegexTokenizer: 규칙 기반으로 단어 만들기. 언어가 바뀌는 부분에서 단어의 경계를 인식한다.

만약 WordExtractor 를 이용하여 단어 점수를 계산하였다면, 단어 점수 중 하나를 택하여 scores 를 만들 수 있다. 아래는 Forward cohesion 의 점수만을 이용하는 경우이다. 그 외에도 다양하게 단어 점수를 정의하여 이용할 수 있다.

## LTokenizer

In [4]:
from soynlp.tokenizer import LTokenizer

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

In [7]:
tokenizer.tokenize("아이오아이는 너무너무너무라는 노래를 불렀어요")

['아이오아이', '는', '너무', '너무너무라는', '노래를', '불렀어요']

명사 추출기의 명사 점수와 Cohesion 을 함께 이용할 수도 있다.. 한 예로, "Cohesion 점수 + 명사 점수"를 단어 점수로 이용하려면 아래처럼 작업할 수 있다.

In [13]:
combined_scores

{'사회복지공동모금회장': 1.0,
 '남북보건의료교육재단': 1.0,
 '대전사이언스페스티벌': 1.0,
 '상호출자제한기업집단': 1.0,
 '부산섬유패션정책포럼': 1.0,
 '공주시립박찬호야구장': 1.0,
 '서울남부고용노동지청': 1.0,
 '한국과학기술연구원장': 1.0,
 '미래창조과학부장관상': 1.0,
 '한국가구수출협동조합': 1.0,
 '한국농수산식품유통공': 1.0,
 '한국폭력학대예방협회': 0.8333333333333334,
 '10억2000만달러': 1.0,
 '사회적책임경영품질원': 1.0,
 '한국표준과학연구원장': 1.0,
 '연구개발특구진흥재단': 1.0,
 '부산인적자원개발원장': 1.0,
 '131조7000억원': 1.0,
 '1329만1000명': 1.0,
 '소비자종합지원시스템': 1.0,
 '전국금융산업노동조합': 1.0,
 '중부지방고용노동청장': 1.0,
 '가루다인도네시아항공': 1.0,
 '세종푸드트럭페스티벌': 1.0,
 '경제정의실천시민연합': 1.0,
 '전주농생명소재연구원': 1.0,
 '한국여성크리스천클럽': 1.0,
 '343조7000억원': 1.0,
 '337억2300만원': 1.0,
 '270억4600만원': 1.0,
 '121조8964억원': 1.0,
 '489조8000억원': 1.0,
 '10억7000만달러': 1.0,
 '도시재생문화클러스터': 1.0,
 '한국열린사이버대학교': 1.0,
 '아우디폭스바겐코리아': 1.5444771680686054,
 '도서관정보정책위원회': 1.0,
 '충북도선거관리위원회': 1.0,
 '전주시사회복지관협회': 1.0,
 '다우존스산업평균지수': 1.0,
 '마이크로소프트연구소': 1.0,
 '고덕상업업무복합단지': 1.0,
 '울진범군민대책위원회': 1.0,
 '전국이동통신유통협회': 1.0,
 '한국시각장애인연합회': 1.0,
 '리틀도쿄마켓플레이스': 1.0,
 '문경시개발자문위원회': 1.0,
 '경기도노인복지관협회': 1.0,
 '마약류통합관리시

In [15]:
from soynlp.noun import LRNounExtractor_v2
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(sentences) # list of str like

noun_scores = {noun:score.score for noun, score in nouns.items()}
combined_scores = {noun:score + cohesion_score.get(noun, 0)
    for noun, score in noun_scores.items()}
combined_scores = combined_scores.update(
    {subword:cohesion for subword, cohesion in cohesion_score.items()
    if not (subword in combined_scores)}
)

tokenizer = LTokenizer(scores=combined_scores)

In [16]:
tokenizer.tokenize("아이오아이는 너무너무너무라는 노래를 불렀어요")

['아이오아이는', '너무너무너무라는', '노래를', '불렀어요']

## MaxScoreTokenizer

In [17]:
from soynlp.tokenizer import MaxScoreTokenizer

scores = {'파스': 0.3, '파스타': 0.7, '좋아요': 0.2, '좋아':0.5}
tokenizer = MaxScoreTokenizer(scores=scores)

print(tokenizer.tokenize('난파스타가좋아요'))
# ['난', '파스타', '가', '좋아', '요']

print(tokenizer.tokenize('난파스타가 좋아요', flatten=False))
# [[('난', 0, 1, 0.0, 1), ('파스타', 1, 4, 0.7, 3),  ('가', 4, 5, 0.0, 1)],
#  [('좋아', 0, 2, 0.5, 2), ('요', 2, 3, 0.0, 1)]]

['난', '파스타', '가', '좋아', '요']
[[('난', 0, 1, 0.0, 1), ('파스타', 1, 4, 0.7, 3), ('가', 4, 5, 0.0, 1)], [('좋아', 0, 2, 0.5, 2), ('요', 2, 3, 0.0, 1)]]


## RegexTokenizer

In [18]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()

print(tokenizer.tokenize('이렇게연속된문장은잘리지않습니다만'))
# ['이렇게연속된문장은잘리지않습니다만']

print(tokenizer.tokenize('숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠'))
# ['숫자', '123', '이영어', 'abc', '에섞여있으면', 'ㅋㅋ', '잘리겠죠']

['이렇게연속된문장은잘리지않습니다만']
['숫자', '123', '이영어', 'abc', '에섞여있으면', 'ㅋㅋ', '잘리겠죠']


C:\Users\sbh0613\anaconda\lib\site-packages\soynlp\tokenizer\_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))
